# Overview

1. Read reachable node data (including IP address and network type)
2. Annotate netgroups
3. Annotate ASN from old and current ASMAP
4. Identify ASN discrepancies

### Helper functions

In [ ]:
import bz2
import ipaddress
import pandas as pd
from tqdm import tqdm

def load_node_data(date: str):
    """Load node data (node address and network type) from CSV."""
    df = pd.read_csv(f"data/reachable-nodes-{date}.csv")
    num_ipv4 = sum(df.network == 'ipv4')
    num_ipv6 = sum(df.network == 'ipv6')
    num_ip = len(df)
    assert num_ipv4 + num_ipv6 == num_ip, "Inconsistent dichotomy"
    print(f'reachable nodes: ipv4={num_ipv4}, ipv6={num_ipv6}, ipv4/ipv6={num_ip}')
    return df

def load_asmap(date: str):
    """Read ASMAP data."""
    asmap = []
    with bz2.open(f"data/asmap-kartograf-{date}.txt.bz2") as f:
        for line in f.readlines():
            net, asn = line.decode('ascii').strip().split(' ')
            network = ipaddress.ip_network(net)
            netw = int(network.network_address)
            mask = int(network.netmask)
            asmap.append((netw, mask, asn))
    return asmap


def addr_to_netgroup(addr_str: str):
    """Derive netgroup from IP address in line with strategy used in Bitcoin Core."""
    addr = ipaddress.ip_address(addr_str)
    if isinstance(addr, ipaddress.IPv4Address):
        return ipaddress.ip_interface(f'{addr}/16').network
    if addr.exploded.startswith("2001:0470:"):
        return ipaddress.ip_interface(f'{addr}/36').network
    return ipaddress.ip_interface(f'{addr}/32').network

def addr_to_asn(addr_str: str, asmap):
    """Derive ASN from IP Address."""
    ip = ipaddress.ip_address(addr_str)
    ip_int = int(ip)
    for netw, mask, asn in asmap:
        if (ip_int & mask) == netw:
            return asn
    return "unknown"

## Load node data, annotate netgroups and ASN (based on old and current ASMAPs)

In [ ]:
now = "2024-01-25"
old = "2023-02-06"

df = load_node_data(now)
tqdm.pandas(desc="annotating netgroups"); df['netgroup'] = df.address.progress_apply(addr_to_netgroup)

asmap_old = asmap = load_asmap(old)
tqdm.pandas(desc="annotating ASN (old)"); df['asn_old'] = df.address.progress_apply(addr_to_asn, args=(asmap_old,))

asmap_now = asmap = load_asmap(now)
tqdm.pandas(desc="annotating ASN (new)"); df['asn_now'] = df.address.progress_apply(addr_to_asn, args=(asmap_now,))

In [ ]:
changed = df[df['asn_old'] != df['asn_now']]
num_changed, num_total = len(changed), len(df)
ratio = num_changed/num_total
print(f"{num_changed} of {num_total} nodes have differing ASN (ratio={ratio*100:.1f}%)")